# Imports


In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo

# Others
import os
import shutil
import zipfile

%matplotlib inline

# Create Tiles

In [4]:
input_file_post = '../Datasets/Post_Event_San_Juan.tif'
input_file_pre = '../Datasets/Pre_Event_San_Juan.tif'
output_dir_post = "../Post_Event_Grids_In_TIFF"
output_dir_pre = "../Pre_Event_Grids_In_TIFF"
grid_x = 512
grid_y = 512

In [5]:
def generate_tiles(input_file, output_dir,grid_x,grid_y):
    ds = gdal.Open(input_file)

    # Get image size and number of bands
    width = ds.RasterXSize
    height = ds.RasterYSize
    num_bands = ds.RasterCount

    # Calculate number of tiles in each dimension
    num_tiles_x = (width // grid_x)
    num_tiles_y = (height // grid_y)

    print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over each tile and save as a separate TIFF image
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            x_offset = i *  grid_x
            y_offset = j *  grid_y

            tile_width = min(grid_x, width - x_offset)
            tile_height = min(grid_y, height - y_offset)

            tile = []
            for band in range(1, num_bands + 1):
                tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
                tile.append(tile_data)

            # Create output filename
            output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
                
            # Create an output TIFF file with same CRS and band values range
            driver = gdal.GetDriverByName("GTiff")
            options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
            out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                       ds.GetRasterBand(1).DataType, options=options)
            # out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, ds.GetRasterBand(1).DataType)

            # Set the geotransform
            geotransform = list(ds.GetGeoTransform())
            geotransform[0] = geotransform[0] + x_offset * geotransform[1]
            geotransform[3] = geotransform[3] + y_offset * geotransform[5]
            out_ds.SetGeoTransform(tuple(geotransform))

            # Set the projection
            out_ds.SetProjection(ds.GetProjection())

            # Write each band to the output file
            for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])

            # Close the output file
            out_ds = None

    print("Tiles generation completed.")

In [6]:
generate_tiles(input_file_post, output_dir_post, grid_x, grid_y)
generate_tiles(input_file_pre, output_dir_pre, grid_x, grid_y)

Total number of tiles: 10730
Tiles generation completed.
Total number of tiles: 10730
Tiles generation completed.


# Convert tiles to jpg for labelling compatability

In [7]:
def convert_tiff_to_jpeg(input_dir,output_dir):
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(input_dir, filename))
        
            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')
        
            # create new filename, replace .tif with .jpg
            output_filename = os.path.splitext(filename)[0] + '.jpg'
        
            # save the image in JPEG format
            print(os.path.join(output_dir, output_filename))
            img.save(os.path.join(output_dir, output_filename), 'JPEG')
    print("Conversion from TIFF to JPEG completed.")

In [8]:
output_dir_post_jpg = "../Post_Event_Grids_In_JPG"
output_dir_pre_jpg = "../Pre_Event_Grids_In_JPG"
input_dir_post = "../Post_Event_Grids_In_TIFF"
input_dir_pre = "../Pre_Event_Grids_In_TIFF"

convert_tiff_to_jpeg(input_dir_post, output_dir_post_jpg)
convert_tiff_to_jpeg(input_dir_pre, output_dir_pre_jpg)

../Post_Event_Grids_In_JPG\tile_0_0.jpg
../Post_Event_Grids_In_JPG\tile_0_1.jpg
../Post_Event_Grids_In_JPG\tile_0_10.jpg
../Post_Event_Grids_In_JPG\tile_0_100.jpg
../Post_Event_Grids_In_JPG\tile_0_101.jpg
../Post_Event_Grids_In_JPG\tile_0_102.jpg
../Post_Event_Grids_In_JPG\tile_0_103.jpg
../Post_Event_Grids_In_JPG\tile_0_104.jpg
../Post_Event_Grids_In_JPG\tile_0_105.jpg
../Post_Event_Grids_In_JPG\tile_0_106.jpg
../Post_Event_Grids_In_JPG\tile_0_107.jpg
../Post_Event_Grids_In_JPG\tile_0_108.jpg
../Post_Event_Grids_In_JPG\tile_0_109.jpg
../Post_Event_Grids_In_JPG\tile_0_11.jpg
../Post_Event_Grids_In_JPG\tile_0_110.jpg
../Post_Event_Grids_In_JPG\tile_0_111.jpg
../Post_Event_Grids_In_JPG\tile_0_112.jpg
../Post_Event_Grids_In_JPG\tile_0_113.jpg
../Post_Event_Grids_In_JPG\tile_0_114.jpg
../Post_Event_Grids_In_JPG\tile_0_115.jpg
../Post_Event_Grids_In_JPG\tile_0_116.jpg
../Post_Event_Grids_In_JPG\tile_0_117.jpg
../Post_Event_Grids_In_JPG\tile_0_118.jpg
../Post_Event_Grids_In_JPG\tile_0_119.jp

In [10]:
def rename_files(directory_path, prefix):
# Define the directory path where your files are located
    directory_path = directory_path
    
    # Get a list of all files in the directory
    files = os.listdir(directory_path)
    
    # Start the numbering from 1
    number = 0
    
    # Loop through each file in the directory
    for filename in files:
        # Check if the item is a file (not a directory)
        if os.path.isfile(os.path.join(directory_path, filename)):
            # Get the file extension
            file_extension = os.path.splitext(filename)[1]
    
            # Create the new file name with leading zeros
            new_filename = f"{prefix}{number:03}{file_extension}"
    
            # Construct the full path to the original and new files
            old_filepath = os.path.join(directory_path, filename)
            new_filepath = os.path.join(directory_path, new_filename)
    
            # Rename the file
            os.rename(old_filepath, new_filepath)
    
            # Increment the number for the next file
            number += 1
    
    print("Files renamed successfully.")

In [11]:
rename_files(output_dir_post_jpg, "Post_Event_")
rename_files(output_dir_pre_jpg, "Pre_Event_")

Files renamed successfully.
Files renamed successfully.


# Coordinates

In [8]:
from osgeo import gdal

# Open the raster dataset
ds = gdal.Open(input_file_post)

# Assuming we know the spatial resolution is 10 meters
resolution = 0.3  # meters

# Get raster dimensions
raster_x_size = ds.RasterXSize  # width in pixels
raster_y_size = ds.RasterYSize  # height in pixels

# Calculate geographic coverage
width_in_meters = raster_x_size * resolution
height_in_meters = raster_y_size * resolution

print(f"Geographic coverage: {width_in_meters}m x {height_in_meters}m")


Geographic coverage: 11477.699999999999m x 22380.6m


# Show the Tiling

In [13]:
def load_tiff_dimensions(image_path):
    with rasterio.open(image_path) as src:
        # Get the dimensions of the TIFF file
        img_width, img_height = src.width, src.height
        
        return img_width, img_height

# Replace 'image_path.tif' with the path to your GeoTIFF file
img_width, img_height = load_tiff_dimensions(input_file_post)
print(f"TIFF dimensions: {img_width} x {img_height}")


TIFF dimensions: 38259 x 74602


In [2]:
def convert_coordinates_to_tiff_space(x, y, geographic_coverage, tiff_dimensions):
    geographic_width, geographic_height = geographic_coverage
    tiff_width, tiff_height = tiff_dimensions
    
    # Calculate the scaling factors
    x_scale = tiff_width / geographic_width
    y_scale = tiff_height / geographic_height
    
    # Convert the coordinates to TIFF space
    tiff_x = int(x * x_scale)
    tiff_y = int(y * y_scale)
    
    return tiff_x, tiff_y


In [19]:
""""
6260.256050  6022.519245
4588.879052  3660.092141
4522.345390  5585.711295
6358.288740  3940.047788
"""

pre_storm_matplot_coords = []

pre_storm_matplot_coords.append(convert_coordinates_to_tiff_space(6260.256050, 6022.519245, (11477.699999999999, 22380.6), (38259, 74602)))
pre_storm_matplot_coords.append(convert_coordinates_to_tiff_space(4588.879052, 3660.092141, (11477.699999999999, 22380.6), (38259, 74602)))
pre_storm_matplot_coords.append(convert_coordinates_to_tiff_space(4522.345390, 5585.711295, (11477.699999999999, 22380.6), (38259, 74602)))
pre_storm_matplot_coords.append(convert_coordinates_to_tiff_space(6358.288740, 3940.047788, (11477.699999999999, 22380.6), (38259, 74602)))
                 
print(pre_storm_matplot_coords)

[(20867, 20075), (15296, 12200), (15074, 18619), (21194, 13133)]


In [3]:
# Post-storm coordinates converted

post_storm_matplot_coords = []

post_storm_matplot_coords.append(convert_coordinates_to_tiff_space(6828, 10778, (11477.699999999999, 22380.6), (38259, 74602)))
post_storm_matplot_coords.append(convert_coordinates_to_tiff_space(7884, 10915, (11477.699999999999, 22380.6), (38259, 74602)))
post_storm_matplot_coords.append(convert_coordinates_to_tiff_space(7726, 9291, (11477.699999999999, 22380.6), (38259, 74602)))
post_storm_matplot_coords.append(convert_coordinates_to_tiff_space(6835, 9455, (11477.699999999999, 22380.6), (38259, 74602)))

print(post_storm_matplot_coords)

[(22760, 35926), (26280, 36383), (25753, 30970), (22783, 31516)]
